In [1]:
# Import
from sklearn.cluster import MeanShift
import numpy as np
import csv
import pandas as pd
from scipy.spatial import distance

In [2]:
# To format a csv file
# with open('checkins.dat') as dat_file, open('file.csv', 'w') as csv_file:
#     csv_writer = csv.writer(csv_file)

#     for line in dat_file:
#         row = [field.strip() for field in line.split('|')]
#         if len(row) == 6 and row[3] and row[4]:
#             csv_writer.writerow(row)

In [3]:
# Reading dataset into DF, creating training sample
df = pd.read_csv('file.csv')
df_train = df.iloc[:100000]

In [4]:
# Creating model and training it
alg = MeanShift(bandwidth=0.1).fit(df_train[['latitude', 'longitude']].values)
# Getting predicts
predicts = alg.predict(df[['latitude', 'longitude']].values)
# Searching for clusters by the number of checks > 15
df['Cluster'] = predicts
checkout_num_in_clust = df.reset_index().groupby('Cluster')['index'].count()
checkout_num_in_clust = checkout_num_in_clust[checkout_num_in_clust > 15]
correct_clust = checkout_num_in_clust.index.values

In [27]:
# Getting cluster centers
clust_centers = alg.cluster_centers_
df_centers = pd.DataFrame(clust_centers, columns=['latitude', 'longitude'])
# Filtering clusters and coords
df_suitable_clust = df_centers[df_centers.index.isin(correct_clust)]

In [28]:
# Offices coords from Google
coord_offices = np.array([[33.751277, -118.188740],
                          [25.867736, -80.324116],
                          [51.503016, -0.075479],
                          [52.378894, 4.885084],
                          [39.366487, 117.036146],
                          [-33.868457, 151.205134]])
# Calculating distances between every clust and every office
clust_dist = distance.cdist(df_suitable_clust.values, coord_offices)
# Calculating min distance for every clust
df_suitable_clust.loc[:,'min dist to office'] = list(map(lambda x: x.min(), clust_dist))
# Searching for the 20 places closest to offices
top_20_clust = df_suitable_clust.sort_values('min dist to office').head(20)
# Searching for the closest clust
final_res = str((top_20_clust.head(1)).values[:,:2])[2:-2]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [134]:
# Writting answer
with open('res_clust_1.txt', 'w') as f:
    f.write(final_res)